In [85]:
import sys


# def extract_lines_with_ids(input_file, output_file, id_array):
#     with open(input_file, 'r') as f_input, open(output_file, 'w') as f_output:
#         for line in f_input:
#             line_id = line.split()[1]  # Assuming the ID is the first word in each line
#             if line_id in id_array:
#                 f_output.write(line)

def extract_lines_with_ids(input_file, output_file, id_array):
    with open(input_file, 'r') as f_input, open(output_file, 'w') as f_output:
        for line in f_input:
            words = line.split()
            if len(words) >= 2:  # Check if the line has at least two words
                line_id = words[1]  # Assuming the ID is the second word in each line
                if (line_id in id_array and words[0]=='ATOM'):
                    f_output.write(line)
        # f_output.write('END')


def distance(coord1, coord2):
    return ((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2 ) ** 0.5

def nearest_neighbor(nodes, coordinates):
    unvisited = set(nodes)
    start_node = nodes[0]
    current_node = start_node
    path = [current_node]
    total_distance = 0
    
    while unvisited:
        nearest_node = None
        min_distance = sys.maxsize
        
        for neighbor in unvisited:
            if neighbor != current_node:
                dist = distance(coordinates[current_node], coordinates[neighbor])
                if dist < min_distance:
                    min_distance = dist
                    nearest_node = neighbor
        
        path.append(nearest_node)
        unvisited.remove(nearest_node)
        total_distance += min_distance
        current_node = nearest_node
    
    total_distance += distance(coordinates[current_node], coordinates[start_node])
    # path.append(start_node)
    
    return total_distance, path



In [86]:

# Example usage:
nodes = ['A', 'B', 'C', 'D', 'E']
coordinates = {'A': (0, 0), 'B': (1, 2), 'C': (3, 1), 'D': (2, 3), 'E': (4, 4)}

min_cost, path = nearest_neighbor(nodes, coordinates)

print("Minimum cost using nearest neighbor:", min_cost)
print("Optimal path:", path)


Minimum cost using nearest neighbor: 12.210904837709434
Optimal path: ['A', 'B', 'D', 'E', 'C', 'A']


In [87]:

# Example usage:
input_file = '4dfr.pdb'
output_file = 'extracted_lines.pdb'
id_array = ['1', '2', '3']  # Add your IDs here

extract_lines_with_ids(input_file, output_file, id_array)


In [88]:

# Example usage:
input_file = '4dfr.pdb'
output_file = 'extracted_lines.pdb'
id_array = ['1', '2', '3']  # Add your IDs here

extract_lines_with_ids(input_file, output_file, id_array)


In [89]:
# 6,7,8

def extract_words(input_file):
    extracted_words = {}
    extracted_aa = {}
    with open(input_file, 'r') as f_input:
        for line in f_input:
            words = line.split()
            if len(words) >= 8:  # Ensure the line has at least 8 words
                extracted_aa[int(words[1])] = (words[3])
                extracted_words[int(words[1])] = (float(words[6]), float(words[7]), float(words[8]))  # Extract words 2nd, 6th, 7th, and 8th
                # extracted_words.append(words_to_extract)
    return extracted_words,extracted_aa

# Example usage:
input_file = 'extracted_lines.pdb'  # Replace 'your_text_file.txt' with the path to your text file
extracted_words,extracted_aa = extract_words(input_file)

# Print the extracted words
# for words in extracted_words:
#     print(words)
print(extracted_words)
print(extracted_aa)

{1: (24.293, 59.579, 4.215), 2: (25.127, 58.554, 4.958), 3: (24.186, 58.457, 6.297)}
{1: 'MET', 2: 'MET', 3: 'MET'}


In [90]:

# Example usage:
nodes = [1,2,3]
coordinates = extracted_words

min_cost, path = nearest_neighbor(nodes, coordinates)

print("Minimum cost using nearest neighbor:", min_cost)
print("Optimal path:", path)
print(coordinates)


Minimum cost using nearest neighbor: 3.3945081808083533
Optimal path: [1, 3, 2, 1]
{1: (24.293, 59.579, 4.215), 2: (25.127, 58.554, 4.958), 3: (24.186, 58.457, 6.297)}


In [133]:
def interpolate_coordinates(point_A, point_B, distance):
    # Calculate the distance between points A and B
    distance_AB = ((point_A[0] - point_B[0])**2 + (point_A[1] - point_B[1])**2) ** 0.5
    
    # Calculate the fraction of distance between A and B
    fraction = distance / distance_AB
    
    # Interpolate coordinates based on the fraction
    interpolated_x = round(point_A[0] + fraction * (point_B[0] - point_A[0]),3)
    interpolated_y = round(point_A[1] + fraction * (point_B[1] - point_A[1]),3)
    interpolated_z = round(point_A[2] + fraction * (point_B[2] - point_A[2]),3)
    
    return (interpolated_x, interpolated_y, interpolated_z)


def add_ala(path,coordinates,threshold):
    final_path=[]
    index = 0
    for i in range(len(path)-1):
        final_path.append(i)
        if (distance(coordinates[path[i]],coordinates[path[i+1]])>threshold):
            coordinates[index-1]=interpolate_coordinates(coordinates[path[i]],coordinates[path[i+1]],threshold)
            final_path.append(index-1)
            index-=1
            while(distance(coordinates[index],coordinates[path[i+1]])>threshold):
                coordinates[index-1]=interpolate_coordinates(coordinates[index],coordinates[path[i+1]],threshold)
                final_path.append(index-1)
                index-=1
    
    return final_path,coordinates

In [134]:
final_path,coordinates= add_ala(path,coordinates,1)

In [135]:
print(final_path)
print(coordinates[-1][0])

[0, -1, 1, 2, -2]
24.198


In [148]:
def ala(i,x,y,z):
    txt = f"ATOM      {i}  C   ALA A   1      {x}  {y}   {z}  1.00 35.50           C\n"
    return txt


def rearrange_lines(input_file, arranged_ids, output_file,coordinates):
    # Read the contents of the text file and store them in memory
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Create a dictionary mapping each ID to its corresponding line
    id_to_line = {}
    for line in lines:
        parts = line.strip().split()
        # print(parts)
        id_ = parts[1]  # Assuming the ID is the second element in each line
        id_to_line[str(id_)] = line  # Convert the ID to string before using it as key
    
    # print(id_to_line)

    # Iterate through the array of arranged IDs and append the corresponding lines to a new list
    rearranged_lines = []
    for id_ in arranged_ids:
        if(id_+1 <= 0):
            rearranged_lines.append(ala(id_,coordinates[id_][0],coordinates[id_][1],coordinates[id_][2]))
        if str(id_+1) in id_to_line:
            rearranged_lines.append(id_to_line[str(id_+1)])

    # Write the lines from the new list to a new text file
    with open(output_file, 'w') as f:
        f.writelines(rearranged_lines)


def replace_ids_with_line_numbers(input_file, output_file):
    # Open the input file for reading
    with open(input_file, 'r') as f_input:
        # Open the output file for writing
        with open(output_file, 'w') as f_output:
            # Initialize line number
            line_number = 1
            # Iterate over each line in the input file
            for line in f_input:
                # Split the line by whitespace and replace the first element (ID) with line number
                parts = line.strip().split()
                parts[1] = str(line_number)
                # Write the modified line to the output file
                f_output.write(' '.join(parts) + '\n')
                # Increment the line number
                line_number += 1
            


In [149]:
input_file = 'extracted_lines.pdb'
arranged_ids = final_path  # Example arranged IDs
output_file = 'lines.pdb'
alanine = 'ALA\n'

rearrange_lines(input_file, arranged_ids, output_file,coordinates)
replace_ids_with_line_numbers('lines.pdb','final.pdb')